In [1]:
import sys
import pandas as pd
import numpy as np
from IPython.display import display
# do not truncate the dataframe column width
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None

sys.path.append('F:/repo/gpsbeam')

from src.config.data_config import DataConfig
from src.config.cnn_ed_rnn_model_config import ModelConfig
from src.config.experiment_config import ExperimentConfig
from src.experiment import RunExperiment

def combined_metric(row):
    loss_db = np.mean(row['pred0_top1_mean_power_loss_db'] + row['pred1_top1_mean_power_loss_db']\
             + row['pred2_top1_mean_power_loss_db'] + row['pred3_top1_mean_power_loss_db'])
    accuracy = np.mean(row['pred0_top1_test_acc_percent'] + row['pred1_top1_test_acc_percent']\
                      + row['pred2_top1_test_acc_percent'] + row['pred3_top1_test_acc_percent']) / 100
    invert_loss_db = 1/loss_db
    return (invert_loss_db * 0.5) + (accuracy * 0.5)

def df_apply_combined_metric(df):
    key = 'combined_loss_db_and_acc'
    df[key] = df.apply(combined_metric, axis=1)
    df = df.sort_values(by=key, ascending=False)
    return df


# Sequence Length Variation

In [2]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=['unit2_loc_minmax_norm', 'unit2to1_vector'],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_seq_len',
                  exp_dict={
                       "seq_len": [3, 4, 5, 6, 7, 8, 9],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:10:40.645 | DEBUG    | src.experiment:run:40 - updated model_config | seq_len to 3
2025-05-20 15:10:40.646 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:10:40.646 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:10:40.646 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2_loc_minmax_norm', 'unit2to1_vector']
2025-05-20 15:11:01.822 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_adjusted.hkl
2025-05-20 15:11:01.823 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method            

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_11_01\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_11_01\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:14:27.110 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:14:27.127 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_11_01\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_11_01\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 64.83it/s]
2025-05-20 15:14:27.209 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.745 0.761 0.767 0.727]
 [0.95  0.969 0.97  0.955]
 [0.991 0.994 0.992 0.986]]
2025-05-20 15:14:32.450 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_11_01\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:14:32.646 

...

test: 100%|██████████| 3/3 [00:00<00:00, 80.58it/s]
2025-05-20 15:25:34.530 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.761 0.753 0.732 0.709]
 [0.973 0.978 0.974 0.953]
 [0.988 0.993 0.991 0.982]]
2025-05-20 15:25:39.168 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_23_25\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:25:39.322 

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_25_39\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_25_39\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:27:49.882 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:27:49.890 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_25_39\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_25_39\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 2/2 [00:00<00:00, 47.15it/s]
2025-05-20 15:27:49.979 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.77  0.771 0.75  0.721]
 [0.973 0.982 0.976 0.959]
 [0.986 0.991 0.987 0.979]]
2025-05-20 15:27:54.784 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\05-20-2025_15_25_39\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:27:54.972 

In [6]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151040\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,seq_len,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
0,3,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,2.078,0.99,0.279,0.298,0.264,0.420,1.713,1.690,1.690,1.685,94.645,94.719,94.718,94.736,1.867,1.836,1.839,1.848,94.166,94.264,94.253,94.226,2.098,2.078,2.087,2.099,93.444,93.506,93.477,93.439,2.525,2.493,2.524,2.530,92.108,92.209,92.113,92.093,3.999,4.019,4.051,4.092,87.503,87.441,87.340,87.214,94.018,96.527,96.961,94.838,98.553,99.228,99.083,98.408,99.759,99.662,99.469,99.276,74.5,95.0,99.1,76.1,96.9,99.4,76.7,97.0,99.2,72.7,95.5,98.6,1.896511
1,4,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,1.531,0.99,0.287,0.232,0.227,0.492,1.694,1.702,1.700,1.704,94.707,94.682,94.686,94.674,1.841,1.819,1.822,1.833,94.245,94.315,94.306,94.271,2.094,2.080,2.084,2.106,93.457,93.499,93.488,93.420,2.459,2.469,2.506,2.563,92.315,92.284,92.168,91.990,3.845,3.975,4.049,4.112,87.984,87.577,87.346,87.149,94.226,96.735,96.735,94.368,98.486,99.432,99.243,98.391,99.763,99.669,99.574,99.243,73.3,96.7,99.2,76.1,98.4,99.5,74.7,98.0,99.5,71.7,96.5,98.8,1.882877
2,5,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,1.583,0.99,0.237,0.236,0.230,0.492,1.649,1.642,1.668,1.682,94.846,94.870,94.787,94.742,1.816,1.787,1.829,1.854,94.325,94.414,94.285,94.205,2.063,2.025,2.048,2.073,93.553,93.673,93.599,93.522,2.450,2.428,2.472,2.489,92.342,92.414,92.276,92.221,3.834,3.745,3.819,3.874,88.019,88.296,88.065,87.895,95.

# Feature Variation


In [3]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=['unit2_loc_minmax_norm', 'unit2to1_vector'],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_feature_input',
                  exp_dict={
                       "model_input_column_list": [
                           ['unit2_loc_minmax_norm'], 
                           ['unit2to1_vector'],
                           ['unit2_loc_minmax_norm', 'unit2to1_vector'],        
                        #    ['unit1_beam_idx_32']
                           ],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:27:55.338 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2_loc_minmax_norm']
2025-05-20 15:27:55.338 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:27:55.339 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:28:17.130 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_adjusted.hkl
2025-05-20 15:28:17.130 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                                : adjusted
                    Portion Percentage                              : 100
           

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_28_17\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_28_17\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:30:16.624 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:30:16.640 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_28_17\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_28_17\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 85.57it/s]
2025-05-20 15:30:16.726 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.648 0.637 0.634 0.625]
 [0.898 0.903 0.897 0.882]
 [0.967 0.971 0.969 0.966]]
2025-05-20 15:30:21.263 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_28_17\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:30:2

...

test: 100%|██████████| 3/3 [00:00<00:00, 85.16it/s]
2025-05-20 15:32:23.223 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.656 0.653 0.649 0.635]
 [0.928 0.945 0.946 0.935]
 [0.969 0.977 0.982 0.979]]
2025-05-20 15:32:27.757 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_30_21\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:32:2

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_32_27\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_32_27\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:34:39.120 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:34:39.127 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_32_27\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_32_27\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 89.39it/s]
2025-05-20 15:34:39.188 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.761 0.753 0.732 0.709]
 [0.973 0.978 0.974 0.953]
 [0.988 0.993 0.991 0.982]]
2025-05-20 15:34:43.910 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\05-20-2025_15_32_27\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:34:4

In [7]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152755\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,model_input_column_list,num_classes,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
2,"['unit2_loc_minmax_norm', 'unit2to1_vector']",32,23,1.475,0.99,0.299,0.278,0.272,0.567,1.560,1.527,1.533,1.542,95.127,95.227,95.209,95.180,1.688,1.660,1.689,1.718,94.726,94.811,94.723,94.631,1.896,1.844,1.863,1.890,94.075,94.236,94.177,94.093,2.271,2.210,2.252,2.294,92.902,93.093,92.962,92.831,3.320,3.319,3.430,3.496,89.625,89.630,89.280,89.075,95.366,95.268,95.707,94.341,99.171,99.024,98.927,98.390,99.659,99.659,99.512,99.220,76.1,97.3,98.8,75.3,97.8,99.3,73.2,97.4,99.1,70.9,95.3,98.2,1.830607
0,['unit2_loc_minmax_norm'],32,23,3.235,0.99,0.726,0.805,0.889,1.224,1.833,1.838,1.838,1.838,94.273,94.256,94.256,94.256,2.029,2.010,2.033,2.038,93.660,93.718,93.646,93.633,2.302,2.248,2.259,2.280,92.806,92.976,92.942,92.877,2.836,2.781,2.802,2.820,91.139,91.309,91.244,91.186,4.180,4.171,4.225,4.270,86.939,86.966,86.797,86.657,90.146,90.293,91.268,90.488,96.195,96.293,96.537,96.098,98.634,98.390,98.195,97.854,64.8,89.8,96.7,63.7,90.3,97.1,63.4,89.7,96.9,62.5,88.2,96.6,1.409212
1,['unit2to1_vector'],32,23,2.771,0.99,1.471,1.365,1.301,1.538,2.081,2.070,2.075,2.089,93.495,93.532,93.515,93.473,2.358,2.327,2.344,2.368,92.631,92.729,92.675,92.599,2.693,2.658,2.658,2.661,91.585,91.695,91.695,91.684,3.310,3.279,3.301,3.310,89.655,89.755,89.683,89.657,4.739,4.804,4.839,4.871,85.191,84.986,84.878,84.779,85.415,86.049,86.537,85.951,93.415,93.610,93.415,93.268,95.951,96

# Var Dataset Split Method

In [4]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=['unit2_loc_minmax_norm', 'unit2to1_vector'],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_spliting_method',
                  exp_dict={
                       "splitting_method": [
                           "sequential",
                           "adjusted",
                           ],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:34:44.410 | DEBUG    | src.experiment:run:36 - updated data_config |splitting_method to sequential
2025-05-20 15:34:44.410 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:34:44.410 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:34:44.410 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2_loc_minmax_norm', 'unit2to1_vector']
2025-05-20 15:34:44.410 | DEBUG    | src.data.dataprep:_check_path_availability:80 - File/Folder f:/repo/gpsbeam\data/raw/Scenario23/scenario23.csv is available.
2025-05-20 15:34:47.030 | DEBUG    | src.preprocessor.gpsprep:__init__:8 - GpsPrep Initialized.
2025-05-20 15:35:12.008 | INFO     | src.data.dataprep:_generate_train_val_test_dataset:335 - Generating train, validation and test datasets...
2025-05-20 15:35:26.119 | INFO     | src.data.dataprep:_save_to_hickle:589 - 
Train-Test dataset is SAVED as f:/repo/gpsbeam\da

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_35_26\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_sequential.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_35_26\dl_generated/model_measurement/label_distribution_scenario_23_sampling_sequential.pdf


2025-05-20 15:37:37.333 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:37:37.333 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_35_26\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_35_26\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 76.81it/s]
2025-05-20 15:37:37.416 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.418 0.404 0.4   0.396]
 [0.898 0.946 0.965 0.959]
 [0.998 0.998 0.998 0.997]]
2025-05-20 15:37:41.821 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_35_26\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:37

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_38_03\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_38_03\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:40:05.314 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:40:05.314 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_38_03\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_38_03\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 89.88it/s]
2025-05-20 15:40:05.385 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.761 0.753 0.732 0.709]
 [0.973 0.978 0.974 0.953]
 [0.988 0.993 0.991 0.982]]
2025-05-20 15:40:09.484 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\05-20-2025_15_38_03\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:40

In [8]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153444\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,splitting_method,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
1,adjusted,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,1.475,0.99,0.299,0.278,0.272,0.567,1.560,1.527,1.533,1.542,95.127,95.227,95.209,95.180,1.688,1.66,1.689,1.718,94.726,94.811,94.723,94.631,1.896,1.844,1.863,1.890,94.075,94.236,94.177,94.093,2.271,2.210,2.252,2.294,92.902,93.093,92.962,92.831,3.320,3.319,3.43,3.496,89.625,89.630,89.280,89.075,95.366,95.268,95.707,94.341,99.171,99.024,98.927,98.39,99.659,99.659,99.512,99.220,76.1,97.3,98.8,75.3,97.8,99.3,73.2,97.4,99.1,70.9,95.3,98.2,1.830607
0,sequential,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,1.586,0.99,1.062,1.066,1.073,1.101,1.984,1.983,2.038,2.065,93.800,93.804,93.631,93.548,2.224,2.17,2.234,2.273,93.051,93.218,93.019,92.898,2.526,2.480,2.565,2.608,92.108,92.250,91.986,91.852,3.378,3.397,3.427,3.477,89.444,89.386,89.291,89.135,5.311,5.309,5.34,5.256,83.402,83.409,83.313,83.576,76.098,75.509,75.509,75.238,90.131,90.584,90.991,90.81,100.000,99.955,99.864,99.728,41.8,89.8,99.8,40.4,94.6,99.8,40.0,96.5,99.8,39.6,95.9,99.7,0.925225


# Var num_classes


In [5]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=['unit2_loc_minmax_norm', 'unit2to1_vector'],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_num_classes',
                  exp_dict={
                       "num_classes": [8, 16, 32, 64]
                  })).run()


2025-05-20 15:40:09.919 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 8
2025-05-20 15:40:09.919 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 8
2025-05-20 15:40:09.919 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2_loc_minmax_norm', 'unit2to1_vector']
2025-05-20 15:40:31.374 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam8_splitting_method_adjusted.hkl
2025-05-20 15:40:31.374 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                                : adjusted
                    Portion Percentage                              :

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_40_31\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_40_31\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:42:31.410 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 8
2025-05-20 15:42:31.410 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_40_31\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_8_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_40_31\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 2/2 [00:00<00:00, 101.97it/s]
2025-05-20 15:42:31.477 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.88  0.881 0.864 0.859]
 [0.997 0.995 0.995 0.992]
 [1.    1.    1.    0.999]]
2025-05-20 15:42:35.414 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_40_31\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:42:35

...

test: 100%|██████████| 3/3 [00:00<00:00, 59.49it/s]
2025-05-20 15:47:33.614 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.761 0.753 0.732 0.709]
 [0.973 0.978 0.974 0.953]
 [0.988 0.993 0.991 0.982]]
2025-05-20 15:47:38.200 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_45_37\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:47:38.

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_48_17\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_48_17\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:50:16.658 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 64
2025-05-20 15:50:16.675 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_48_17\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_64_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_48_17\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 83.08it/s]
2025-05-20 15:50:16.758 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.632 0.64  0.641 0.634]
 [0.913 0.931 0.929 0.91 ]
 [0.974 0.972 0.976 0.97 ]]
2025-05-20 15:50:21.595 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\05-20-2025_15_48_17\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:50:21.

In [9]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154009\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
# df = df_apply_combined_metric(df)
df

,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent
0,8,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,0.349,0.99,0.232,0.343,0.521,0.731,1.194,1.213,1.233,1.248,85.077,84.841,84.593,84.401,1.235,1.257,1.282,1.298,84.562,84.290,83.974,83.775,1.294,1.317,1.350,1.364,83.831,83.540,83.125,82.945,1.443,1.478,1.501,1.530,81.966,81.520,81.235,80.875,1.969,2.024,2.111,2.192,75.390,74.703,73.618,72.602,96.282,95.885,95.141,94.695,98.761,98.265,97.819,97.620,99.603,99.554,99.008,98.810,88.0,99.7,100.0,88.1,99.5,100.0,86.4,99.5,100.0,85.9,99.2,99.9
1,16,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,0.454,0.99,0.292,0.238,0.280,0.407,1.258,1.255,1.279,1.294,92.136,92.155,92.004,91.915,1.318,1.317,1.329,1.352,91.762,91.771,91.691,91.547,1.412,1.391,1.416,1.443,91.176,91.305,91.148,90.982,1.581,1.568,1.621,1.643,90.120,90.200,89.869,89.734,2.240,2.262,2.362,2.437,86.000,85.862,85.236,84.766,95.238,95.876,96.514,95.287,98.527,98.871,98.675,98.135,99.460,99.656,99.558,99.313,84.1,98.6,99.4,86.0,98.8,99.3,86.0,99.0,99.6,84.3,98.5,99.5
2,32,"['unit2_loc_minmax_norm', 'unit2to1_vector']",23,1.475,0.99,0.299,0.278,0.272,0.567,1.560,1.527,1.533,1.542,95.127,95.227,95.209,95.180,1.688,1.660,1.689,1.718,94.726,94.811,94.723,94.631,1.896,1.844,1.863,1.890,94.075,94.236,94.177,94.093,2.271,2.210,2.252,2.294,92.902,93.093,92.962,92.831,3.320,3.319,3.430,3.496,89.625,89.630,89.280,89.075,95.366,95.268,95.707,94.341,99.171,99.024,98.927,98.390,99